## Populate Inventory Data
The purpose of this workbook is to load-up all the flow information I hand-curated, and assign it to the flow entities.  Then I will use the metadata to assign (random) mass and economic value properties to the flows.

Later I will use the metainfo to terminate fragments using the flows.

In [1]:
from antelope_catalog import LcCatalog

In [2]:
cat = LcCatalog('/data/GitHub/2019/EPA_PSM_antelope/catalog/')

Loading JSON data from /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json:
local.qdb: /data/GitHub/2019/EPA_PSM_antelope/catalog/reference-quantities.json
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
6 new flow entities added (6 total)
25 new quantity entities added (25 total)


In [3]:
cat.show_interfaces()

foreground.psm-hackathon [basic, foreground, index, quantity]
local.ecoinvent.3.4.apos [basic, inventory]
local.ecoinvent.3.4.apos.index.20190528 [basic, index]
local.elcd.3.2 [basic, inventory, quantity]
local.elcd.3.2.index.20190528 [basic, index]
local.qdb [basic, index, quantity]
local.usepa.useeio.1.1 [basic, inventory, quantity]
local.usepa.useeio.1.1.index.20190528 [basic, index]
local.uslci.olca [basic, inventory, quantity]
local.uslci.olca.index.20190528 [basic, index]


In [4]:
fg = cat.create_foreground('psm-hackathon')

In [5]:
ar = cat.get_archive(fg.origin)

foreground.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
Loading /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon
foreground.psm-hackathon: /data/GitHub/2019/EPA_PSM_antelope/catalog/psm-hackathon/entities.json
1019 new flow entities added (1019 total)
0 new fragment entities added (0 total)
1 new quantity entities added (1 total)
1402 new fragment entities added (1402 total)


In [6]:
f = next(ar.entities_by_type('flow'))

In [7]:
from lcatools.entities.xlsx_editor import XlsxArchiveUpdater

In [8]:
f.show()

Flow Entity (ref 3M1169C3D6A)
origin: foreground.psm-hackathon
reference: [local.qdb] Number of items [Item(s)]
       Name: BOLT
    Comment: 
      Level: 1.0
  CasNumber: 
Compartment: []
         PT: 5


In [9]:
with XlsxArchiveUpdater(ar, 'PSM-hackathon-leaf-flows_annotated.xlsx') as x:
    x.apply()

In [10]:
f.show()

Flow Entity (ref 3M1169C3D6A)
origin: foreground.psm-hackathon
reference: [local.qdb] Number of items [Item(s)]
       Name: BOLT
    Comment: 
         PT: 5
      Level: 1.0
  CasNumber: 
Compartment: []
   Material: Nickel
       Size: Small
   PartType: Hardware


In [11]:
ar.save()

In [12]:
import xlrd

def mk_kwd(fname, sheet='mass_hints'):
    _kwd = dict()

    _xl = xlrd.open_workbook(fname)
    _s = _xl.sheet_by_name(sheet)
    for r in range(1, _s.nrows):
        _kwd[_s.cell(r, 0).value.lower()] = {'mass': _s.cell(r, 1).value, 'price': _s.cell(r, 2).value}
    return _kwd
    


In [13]:
mass = cat.lcia_engine.get_canonical('mass')
USD = next(cat.query('local.usepa.useeio').quantities(Name='price'))

Loading JSON data from /data/GitHub/2019/EPA_PSM_antelope/catalog/index/0bfdaea6ff0149c86e4d245afec4b2e47c07afb8.json.gz:
local.usepa.useeio.1.1.index.20190528: /data/GitHub/2019/EPA_PSM_antelope/catalog/index/0bfdaea6ff0149c86e4d245afec4b2e47c07afb8.json.gz
local.usepa.useeio.1.1: /data/LCI/USEEIO/USEEIOv1.1-JSONLD-foropenlca.zip
388 new process entities added (388 total)
2261 new flow entities added (2261 total)
24 new quantity entities added (24 total)
Missing canonical quantity-- adding to LciaDb
registering local.usepa.useeio.1.1/b0682037-e878-4be4-a63a-a7a81053a691
local.usepa.useeio.1.1: /data/LCI/USEEIO/USEEIOv1.1-JSONLD-foropenlca.zip
Found Extension: zip
Applying stored configuration
Applying context hint local.usepa.useeio.1.1:water => to water
Applying context hint local.usepa.useeio.1.1:air => to air
Applying configuration to OpenLcaJsonLdArchive with 0 entities at /data/LCI/USEEIO/USEEIOv1.1-JSONLD-foropenlca.zip
Retrieving canonical [local.usepa.useeio.1.1] Producer Pric

In [14]:
USD.show()

QuantityRef catalog reference (b0682037-e878-4be4-a63a-a7a81053a691)
origin: local.usepa.useeio.1.1
UUID: b0682037-e878-4be4-a63a-a7a81053a691
   Name: Producer Price US 2013


In [15]:
kwd = mk_kwd('PSM-hackathon-leaf-flows_annotated.xlsx')

In [16]:
import random

In [17]:
help(random.lognormvariate)

Help on method lognormvariate in module random:

lognormvariate(mu, sigma) method of random.Random instance
    Log normal distribution.
    
    If you take the natural logarithm of this distribution, you'll get a
    normal distribution with mean mu and standard deviation sigma.
    mu can have any value, and sigma must be greater than zero.



In [18]:
random.lognormvariate(0, 0.15)

1.0262747980959792

In [19]:
f.show()

Flow Entity (ref 3M1169C3D6A)
origin: foreground.psm-hackathon
reference: [local.qdb] Number of items [Item(s)]
       Name: BOLT
    Comment: 
         PT: 5
      Level: 1.0
  CasNumber: 
Compartment: []
   Material: Nickel
       Size: Small
   PartType: Hardware


In [20]:
_m_base = -3
_p_base = -3
_sig_base = 0.65


def mock_cfs(flow, kwdict, demo=True):
    m = p = 1.0
    sig = _sig_base
    for k in ('PartType', 'Material', 'Size'):
        if flow.has_property(k):
            sig *= 0.75  # narrow sigma for each specification
            kw = flow[k].lower()
            if kw in kwdict:
                val_d = kwdict[kw]
                m *= val_d['mass']
                p *= val_d['price']
            else:
                if kw is not None:
                    print('missing kwd %s' % kw)
    m *= random.lognormvariate(_m_base, sig)
    p *= random.lognormvariate(_p_base, sig)
    if demo:
        print('[%s] %s: mass %3g %s; price: %3g %s' % (flow.external_ref, flow.name, m, mass.unit(), p, USD.unit()))
    else:
        return m, p
            

In [21]:
help(fg.characterize)

Help on method characterize in module lcatools.interfaces.iquantity:

characterize(flowable, ref_quantity, query_quantity, value, context=None, location='GLO', **kwargs) method of antelope_catalog.catalog_query.CatalogQuery instance
    Add Characterization data for a flowable, reporting the amount of a query quantity that is equal to a unit
    amount of a reference quantity, for a given context and location
    :param flowable: string or flow external ref
    :param ref_quantity: string or external ref
    :param query_quantity: string or external ref
    :param value: float or dict of locations to floats
    :param context: string
    :param location: string, ignored if value is dict
    :param kwargs: overwrite=False, origin=query_quantity.origin, others?
    :return:



In [22]:
cf_dict = {}

In [23]:
m, p = mock_cfs(f, kwd, demo=False)
cf_dict[f.link] = {'mass': m, 'price': p}
cf_dict

{'foreground.psm-hackathon/3M1169C3D6A': {'mass': 0.01120718297564451,
  'price': 0.2896358007847856}}

In [24]:
fg.get(f.external_ref).show()

FlowRef catalog reference (3M1169C3D6A)
origin: foreground.psm-hackathon
   Name: BOLT
Comment: 
==Local Fields==
CasNumber: 


In [25]:
cf = fg.characterize(f.link, f.reference_entity, mass, cf_dict[f.link]['mass'])

In [26]:
cf

Characterization(BOLT, Item(s), kg, None: 0.0112)

In [27]:
cf2 = fg.characterize(f.link, f.reference_entity, USD, cf_dict[f.link]['price'])

In [28]:
cf2

Characterization(BOLT, Item(s), USD, None:   0.29)

In [29]:
[str(k) for k in fg.profile(f)]

['BOLT [context None]: 1 Item(s) x 0.0112072 kg/Item(s) [GLO] (foreground.psm-hackathon)',
 'BOLT [context None]: 1 Item(s) x 0.289636 USD/Item(s) [GLO] (foreground.psm-hackathon)']

In [30]:
cf_dict

{'foreground.psm-hackathon/3M1169C3D6A': {'mass': 0.01120718297564451,
  'price': 0.2896358007847856}}

In [31]:
for f in fg.flows(Material='.'):
    if f.link in cf_dict:
        continue
    m, p = mock_cfs(f, kwd, demo=False)
    cf_dict[f.link] = {'mass': m, 'price': p}
    fg.characterize(f.link, f.reference_entity, mass, cf_dict[f.link]['mass'])
    fg.characterize(f.link, f.reference_entity, USD, cf_dict[f.link]['price'])


missing kwd tiny
missing kwd tiny
missing kwd tiny


In [32]:
fg.save()

True

In [33]:
_flow_gen = fg.flows(Material='.')

In [34]:
f = next(_flow_gen)

In [35]:
[str(k) for k in fg.profile(f)]

['BOLT [context None]: 1 Item(s) x 0.0112072 kg/Item(s) [GLO] (foreground.psm-hackathon)',
 'BOLT [context None]: 1 Item(s) x 0.289636 USD/Item(s) [GLO] (foreground.psm-hackathon)']

In [43]:
f = next(_flow_gen)
print('%s [%s]\nCFs: mass: %.3g %s; price: %.3g %s' % (f, f.external_ref, mass.cf(f), mass.unit(), USD.cf(f), USD.unit()))
assert mass.cf(f) == cf_dict[f.link]['mass']
assert USD.cf(f) == cf_dict[f.link]['price']

[foreground.psm-hackathon] WASHER, FLAT [Item(s)] [4M116-39001]
CFs: mass: 0.00555 kg; price: 2.19 USD


In [44]:
USD.cf(f)

2.1854723935572635

In [45]:
cf_dict[f.link]

{'mass': 0.005553813590308264, 'price': 2.1854723935572635}

In [46]:
f = next(fg.flows(Material='Assembly'))

In [47]:
f.show()

FlowRef catalog reference (74A400503-2001)
origin: foreground.psm-hackathon
   Name: PLATE, IDENT
Comment: 
==Local Fields==
CasNumber: 
 Material: Assembly
     Size: Small


In [48]:
f.has_property('Size')

True

In [49]:
f.show()

FlowRef catalog reference (74A400503-2001)
origin: foreground.psm-hackathon
   Name: PLATE, IDENT
Comment: 
==Local Fields==
CasNumber: 
 Material: Assembly
     Size: Small


In [50]:
print('%s [%s]\nCFs: mass: %.3g %s; price: %.3g %s' % (f, f.external_ref, mass.cf(f), mass.unit(), USD.cf(f), USD.unit()))
assert mass.cf(f) == cf_dict[f.link]['mass']
assert USD.cf(f) == cf_dict[f.link]['price']

[foreground.psm-hackathon] PLATE, IDENT [Item(s)] [74A400503-2001]
CFs: mass: 0.339 kg; price: 4.63 USD
